# Symbolic Algebra (2.5.3)
## Arithmetic on polynomials
Polynomial = a sum of terms each of which is either a:
- Coefficient
- Power of the indeterminate
- Product of a coefficient and a power of the indeterminate

Coefficient = an algebraic expression that is *not dependent* upon the indeterminate of the polynomial.

$5x^2 + 3x +7$ = a polynomial **in** $x$

$(y^2+1)x^3+(2y)x+1$ = a polynomial **in** $x$ whose *coefficients* are polynomials in $y$

In [ ]:
;; from 2.3.2
(define (variable? x) (symbol? x))
(define (same-variable? v1 v2)
  (and (variable? v1)
       (variable? v2)
       (eq? v1 v2)))

(define (install-polynomial-package)
  ;;internal prcedures
  ;; representation of poly
  (define (make-poly variable term-list)
    (cons variable term-list))
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  
  ;;representation of terms and term lists
  ;; term lists are lists of terms, arranged from highest-order to
  ;; lowest-order term
  ;; i.e. ((100 1) (2 2) (0 1)) = x^100 + 2x^2 + 1
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (cons term term-list)))
  (define (the-empty-termlist) '())
  (define (first-term term-list) (car term-list))
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list)
    (null? term-list))
  (define (make-term order coeff)
    (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (add-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               ADD-POLY"
               (list p1 p2))))
  
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1
                     (add-terms (rest-terms L1)
                                L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2
                     (add-terms
                      L1
                      (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term
                      (order t1)
                      (add (coeff t1)
                           (coeff t2)))
                     (add-terms
                      (rest-terms L1)
                      (rest-terms L2)))))))))
  
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (mul-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               MUL-POLY"
               (list p1 p2))))
  
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms
         (mult-term-by-all-terms
          (first-term L1) L2)
         (mul-terms (rest-terms L1) L2))))
  
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term
            (+ (order t1) (order t2))
            (mul (coeff t1) (ceoff t2)))
           (mul-term-by-all-terms
            t1
            (rest-terms L))))))
  
  ;;interface to rest of the system
  (define (tag p) (attach-tag 'polynomial p))
  (put 'add '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (mul-poly p1 p2))))
  (put 'make 'polynomial
       (lambda (var terms)
         (tag (make-poly var terms))))
  'done)

;; generic procedure for users of the polynomial package
(define (make-polynomial var terms)
  ((get 'make 'polynomial) var terms))

## 2.87

In [ ]:
;; polynomial is equal to 0 if it has no terms or each of its terms
;; are equal to 0

;; in polynomial package
(define (terms-list-zero? terms-list)
  (if (empty-termlist? terms-list)
      #t
      (and (=zero? (coeff (first-term terms-list)))
           (terms-list-zero (rest-terms terms-list)))))
(put '=zero? 'polynomial (lambda (p) (terms-list-zero? 
                                      (term-list p))))

## 2.88
Subtraction of polynomials $p_1, p_2$ can be seen as addition of $p_1$ to the *negation* of $p_2$:

$p_1-p_2=p_1+(-p_2)$

Negation of a polynomial is achieved by the negation of each of it's terms:

$-(x^{100} + 2x^2 +1) = -x^{100}-2x^2-1$

In [ ]:
;; generic negation operation
(define (negate x) (apply-generic 'negate x))

;; install negate for other number representations
;; integer package
(put 'negate 'integer (lambda (x) (tag (- x))))

;; rational package
(put 'negate 'rational (lambda (x) (make-rat (- (numer x))
                                             (denom x))))

;; real package
(put 'negate 'real (lambda (x) (tag (- x))))

;; complex package
(put 'negate 'complex (lambda (x) (make-from-real-imag
                                   (negate (real-part x))
                                   (negate (imag-part x)))))

;; polynomial package
;; define negation for polynomial
(define (negate-terms-list L)
  (if (empty-termlist? L)
      (the-empty-termlist)
      (let ((t (first-term L)))
        (adjoin-term
         (make-term (order term) 
                    (negate (coeff term)))
         (negate-terms-list (rest-terms L))))))
(define (negate-poly p)
  (make-poly (variable p)
             (negate-terms-list (term-list p))))
;; define subtraction using negation
(define (sub-poly p1 p2)
  (if (same-variable? (variable p1) (variable p2))
      (make-poly (variable p1)
                 (add-terms (term-list p1)
                            (negate-terms-list p2)))
      (error "Polys not in same var:
             SUB-POLY"
             (list p1 p2))))
(put 'negate 'polynomial (lambda (p) (tag (negate-poly p))))
(put 'sub '(polynomial 'polynomial) 
     (lambda p1 p2) (tag (sub-poly p1 p2)))

## 2.89
Represent dense polynomials as list of coefficeients:

$x^2+5x+1=(1,5,1)$

$2x^3+6x+2=(2,0,6,2)$

Changes required to polynomial package:
- `first-term` needs to construct a 'full term' from the coefficients in the terms list
    - i.e. `'(order coefficient)`
- `adjoin-term` should pad the terms list with zeroes if necessary
    - New term has order greater than 1 more than the current highest order term:
    ```scheme
    (adjoin-term '(4 1) '(2 3 1))
    '(1 0 2 3 1)
    
    (adjoin-term '(4 1) '(1))
    '(1 0 0 0 1)
    ```

In [ ]:
  ;; in polynomial package
  ;; representation of terms and term lists
  (define (first-term term-list)
    (make-term 
     ;; first term is highest order
     (- (length term-list) 1)
     (car term-list)))
  
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (let* ((current-order (order (first-term term-list)))
               (new-order (order term))
               (order-diff (- (current-order new-order))))
          (cond ((= order-diff 1)
                 (cons (coeff term) term-list))
                ((> order-diff 1)
                 (cons (coeff term)
                       (append (zeroes-list (- order-diff 1))
                               (term-list))))
                (else
                 (error "TERM order greater must be greater than
                        highest order term in TERM-LIST:
                        ADJOIN-TERM"
                        (list term term-list)))))))

  (define (zeroes-list n)
    (if (= n 0)
        '()
        (cons 0 (zeroes-list (- n 1)))))

## 2.90
### Complete System So Far

In [ ]:
;; type hierarchy

;; install type-val procedures for each type in type-tower
;; which return corresponding integer hierarchy position
;; based on index in the type-tower
(define (install-type-tower tower)
  (define (iter t index)
    (cond ((null? t)'())
          (else 
           (put 'type-val (car t) (lambda (x) index))
           (iter (cdr t) (+ index 1)))))
  (iter tower 0))

;; find highest type of argument list
(define (highest-type args)
  (define (iter highest-type remaining-args)
    (cond ((null? remaining-args) highest-type)
          (let* ((arg-type (type-tag (car remaining-args)))
                 (arg-val (type-val arg-type))
                 (highest-val (type-val highest-type)))
            (if (> arg-val highest-val)
                (iter arg-type (cdr remaining-args))
                (iter highest-type (cdr remaining-args))))))
  (iter #f args))

;; raise a single value to a given type
(define (raise-to type value)
  (cond ((eq? type (type-tag value)) value)
        ((memq type type-tower) (raise-to type (raise value)))
        (else (error "Cannot raise to type not in type tower RAISE-TO"
                     (list type type-tower)))))

;; raise a list of values to a given type
(define (raise-args-to type args)
  (if (null? args)
      '()
      (cons (raise-to type (car args)) (raise-args-to type (cdr args)))))

;; generic procedures
(define (apply-generic op . args)
  (let* ((type-tags (map type-tag args))
         (proc (get op type-tags)))
    (if proc
        (apply proc (map contents args))
        (if (> (length args) 1)
            (let* ((highest-arg-type (highest-type type-tags))
                   (mapped-args (raise-args-to highest-arg-type args))
                   (mapped-types (map type-tag mapped-args))
                   (mapped-proc (get op mapped-types)))
              (if mapped-proc
                  ;; use drop to 'simplify' results
                  (drop (apply 
                         mapped-proc
                         (map contents mapped-args)))
                  (error
                   "No method for these types -- APPLY-GENERIC"
                   (list op type-tags))))))))

;; from 2.3.2
(define (variable? x) (symbol? x))
(define (same-variable? v1 v2)
  (and (variable? v1)
       (variable? v2)
       (eq? v1 v2)))

(define (real-part z)
  (apply-generic 'real-part z))
(define (imag-part z)
  (apply-generic 'imag-part z))
(define (magnitude z)
  (apply-generic 'magnitude z))
(define (angle z)
  (apply-generic 'angle z))

(define (make-rational n d)
  ((get 'make 'rational) n d))
(define (make-real n)
  ((get 'make 'real) n))
(define (make-complex-from-real-imag x y)
  ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a)
  ((get 'make-from-mag-ang 'complex) r a))
(define (make-polynomial var terms)
  ((get 'make 'polynomial) var terms))

(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))
(define (equ? x y) (apply-generic 'equ? x y))
(define (=zero? x) (apply-generic '=zero? x))
(define (square x) (apply-generic 'square x))
(define (sqr-root x) (apply-generic 'sqr-root x))
(define (sine x) (apply-generic 'sine x))
(define (cosine x) (apply-generic 'square x))
(define (arctan x y) (apply-generic 'arctan x y))
(define (negate x) (apply-generic 'negate x))

;; generic procedure to retrieve the hierarchy of a type
;; as an integer 0 = low
(define (type-val type) (apply-generic 'type-val type))

;; tagging procedures
(define (attach-tag type-tag contents)
  (if (eq? type-tag 'integer)
      contents
      (cons type-tag contents)))

(define (type-tag datum)
  (cond ((number? datum) 'integer)
        ((pair? datum) (car datum))
        (else (error "Bad tagged datum:
                     TYPE-TAG" datum))))

(define (contents datum)
  (cond? ((number? datum) datum)
         ((pair? datum) (cdr datum))
         (error "Bad tagged datum:
                CONTENTS" datum)))

(define (install-integer-package)
  (define (tag x)
    (attach-tag 'integer x))
  (define (int->rational x)
    (make-rational n 1))
  
  (put 'add '(integer integer)
       (lambda (x y) (tag (+ x y))))
  (put 'sub '(integer integer)
       (lambda (x y) (tag (- x y))))
  (put 'mul '(integer integer)
       (lambda (x y) (tag (* x y))))
  (put 'div '(integer integer)
       (lambda (x y) (tag (/ x y))))
  (put '=zero? '(integer) zero?)
  (put 'raise 'integer int->rational)
  (put 'square 'integer (lambda (x) (tag (* x x))))
  (put 'sqr-root 'integer (lambda (x) (make-real (sqrt x))))
  (put 'sine 'integer (lambda (x) (make-real (sin x))))
  (put 'cosine 'integer (lambda (x) (make-real (cos x))))
  (put 'arctan 'integer (lambda (x) (make-real (atan x y))))
  (put 'negate 'integer (lambda (x) (tag (- x))))
  (put 'make 'integer (lambda (x) (tag x)))
  'done)

(define (install-rational-package)
  ;; internal procedures
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (gcd n d)))
      (cons (/ n g) (/ d g))))
  (define (add-rat x y)
    (make-rat (+ (* (numer x) (denom y))
                 (* (numer y) (denom x)))
              (* (denom x) (denom y))))
  (define (sub-rat x y)
    (make-rat (- (* (numer x) (denom y))
                 (* (numer y) (denom x)))
              (* (denom x) (denom y))))
  (define (mul-rat x y)
    (make-rat (* (numer x) (numer y))
              (* (denom x) (denom y))))
  (define (div-rat x y)
    (make-rat (* (numer x) (denom y))
              (* (denom x) (numer y))))
  (define (equ?-rat x y) 
   (and (equ? (numer x) (numer y))
        (equ? (denom x) (denom y))))
  (define (=zero?-rational x)
    (zero? (numer x)))
  (define (rat->real x)
    (make-real (/ numer x) (denom x)))
  (define (rat->integer x)
    (make-integer (/ (numer x) (denom x))))
  (define (rational->ratio x) (/ (numer x) (denom y)))

  ;; interface to rest of the system
  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational)
       (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational)
       (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational)
       (lambda (x y) (tag (mul-rat x y))))
  (put 'div '(rational rational)
       (lambda (x y) (tag (div-rat x y))))
  (put 'equ? '(rational rational) equ?-rat)
  (put '=zero? '(rational) =zero?-rational)
  (put 'project 'rational rat->integer)
  (put 'raise 'rational rat->real)
  (put 'square 'integer (lambda (x) (tag (mul-rat x x))))
  (put 'sqr-root 'integer (lambda (x) (make-real 
                                       (sqrt (rational->ratio x)))))
  (put 'sine 'integer (lambda (x) (make-real 
                                   (sin (rational->ratio x)))))
  (put 'cosine 'integer (lambda (x) (make-real 
                                     (cos (rational->ratio x)))))
  (put 'arctan 'integer (lambda (x) (make-real (atan 
                                                (rational->ratio x) 
                                                (rational->ratio y)))))
  (put 'negate 'rational (lambda (x) (make-rat (- (numer x))
                                               (denom x))))
  (put 'make 'rational
       (lambda (n d) (tag (make-rat n d))))
  'done)

(define (install-complex-package)
  ;; imported procedures from rectangular 
  ;; and polar packages
  (define (make-from-real-imag x y)
    ((get 'make-from-real-imag 
          'rectangular) 
     x y))
  (define (make-from-mag-ang r a)
    ((get 'make-from-mag-ang 'polar) 
     r a))
  ;; internal procedures
  (define (add-complex z1 z2)
    (make-from-real-imag 
     (+ (real-part z1) (real-part z2))
     (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2)
    (make-from-real-imag 
     (- (real-part z1) (real-part z2))
     (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2)
    (make-from-mag-ang 
     (* (magnitude z1) (magnitude z2))
     (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2)
    (make-from-mag-ang 
     (/ (magnitude z1) (magnitude z2))
     (- (angle z1) (angle z2))))
  (define (=zero?-complex z)
    (zero? (magnitude z)))
  (define (equ?-complex z1 z2)
    (and (equ? (magnitude z1) (magnitude z2))
         (equ? (angle z1) (angle z2))))
  (define (complex->real z) (make-real (real-part z)))

  ;; interface to rest of the system
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex)
       (lambda (z1 z2) 
         (tag (add-complex z1 z2))))
  (put 'sub '(complex complex)
       (lambda (z1 z2) 
         (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex)
       (lambda (z1 z2) 
         (tag (mul-complex z1 z2))))
  (put 'div '(complex complex)
       (lambda (z1 z2) 
         (tag (div-complex z1 z2))))
  (put '=zero? 'complex =zero?-complex)
  (put 'real-part 'complex (lambda (x) (real-part x)))
  (put 'imag-part 'complex (lambda (x) (imag-part x)))
  (put 'magnitude 'complex) (lambda (x) (magnitude x)))
  (put 'angle 'complex) (lambda (x) (angle x)))

  (put 'equ? '(complex complex) equ?-complex)
  (put 'project 'complex complex->real)
  (put 'negate 'complex (lambda (x) (make-from-real-imag
                                     (negate (real-part x))
                                     (negate (imag-part x)))))
  (put 'make-from-real-imag 'complex
       (lambda (x y) 
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex
       (lambda (r a) 
         (tag (make-from-mag-ang r a))))
  'done)

(define (install-real-package)
  ;; internal procedures
  (define (tag r) (attach-tag 'real r))
  (define (real->complex r)
    (make-complex-from-real-imag (make-real r)
                                 (make-real 0)))
  ;; using built in scheme real numbers -> utilise numerator/denominator
  ;; and inexact->exact procedures
  (define (real->rational r)
    (make-rational (inexact->exact (numerator r))
                   (inexact->exact (denominator r))))
  
  ;; interface to rest of system
  (put 'add '(real real) (lambda (x y) (tag (+ x y))))
  (put 'sub '(real real) (lambda (x y) (tag (- x y))))
  (put 'mul '(real real) (lambda (x y) (tag (* x y))))
  (put 'div '(real real) (lambda (x y) (tag (/ x y))))
  (put 'square 'integer (lambda (x) (tag (* x x))))
  (put 'sqr-root 'integer (lambda (x) (tag (sqrt x))))
  (put 'sine 'integer (lambda (x) (tag (sin x))))
  (put 'cosine 'integer (lambda (x) (tag (cos x))))
  (put 'arctan 'integer (lambda (x) (tag (atan x y))))
  (put 'equ '(real real) =)
  (put '=zero? 'real (lambda (x) (= x 0)))
  (put 'raise 'real real->complex)
  (put 'project 'real real->rational)
  (put 'negate 'real (lambda (x) (tag (- x))))
  (put 'make 'real (lambda (x)
                     (if (real? x)
                         (tag x)
                         (error "non-real value" x))))
  'done)

(define (install-rectangular-package)  
  ;; internal procedures
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y)
    (cons x y))
  (define (magnitude z)
    (sqr-root (+ (square (real-part z))
                 (square (imag-part z)))))
  (define (angle z)
    (arctan (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a)
    (cons (* r (cosine a)) (* r (sine a))))
  
  ;; interface to the rest of the system
  (define (tag x)
    (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)

(define (install-polar-package)
  ;; internal procedures
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z)
    (* (magnitude z) (cosine (angle z))))
  (define (imag-part z)
    (* (magnitude z) (sine (angle z))))
  (define (make-from-real-imag x y)
    (cons (sqr-root (+ (square x) (square y)))
          (arctan y x)))

  ;; interface to the rest of the system
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)

;; sparse representation of polynomials
;; term lists are lists of terms, arranged from highest-order to
;; lowest-order term
;; i.e. ((100 1) (2 2) (0 1)) = x^100 + 2x^2 + 1
(define (install-sparse-polynomial-package)
  ;;internal procedures
  ;; representation of poly
  (define (make-poly variable term-list)
    (cons variable term-list))
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  
  ;;representation of terms and term lists
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (cons term term-list)))
  (define (the-empty-termlist) '())
  (define (first-term term-list) (car term-list))
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list)
    (null? term-list))
  (define (make-term order coeff)
    (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (add-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               ADD-POLY"
               (list p1 p2))))
  
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1
                     (add-terms (rest-terms L1)
                                L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2
                     (add-terms
                      L1
                      (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term
                      (order t1)
                      (add (coeff t1)
                           (coeff t2)))
                     (add-terms
                      (rest-terms L1)
                      (rest-terms L2)))))))))
  
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (mul-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               MUL-POLY"
               (list p1 p2))))
  
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms
         (mult-term-by-all-terms
          (first-term L1) L2)
         (mul-terms (rest-terms L1) L2))))
  
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term
            (+ (order t1) (order t2))
            (mul (coeff t1) (ceoff t2)))
           (mul-term-by-all-terms
            t1
            (rest-terms L))))))
  
  (define (terms-list-zero? terms-list)
    (if (empty-termlist? terms-list)
        #t
        (and (=zero? (coeff (first-term terms-list)))
             (terms-list-zero (rest-terms terms-list)))))
  (define (negate-terms-list L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t (first-term L)))
          (adjoin-term
           (make-term (order term)
                      (negate (coeff term)))
           (negate-terms-list (rest-terms L))))))
  
  (define (negate-poly p)
    (make-poly (variable p)
               (negate-terms-list (term-list p))))
  (define (sub-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (add-terms (term-list p1)
                              (negate-terms-list p2)))
        (error "Polys not in same var:
               SUB-POLY"
               (list p1 p2))))

  ;;interface to rest of the system
  (define (tag p) (attach-tag 'polynomial p))
  (put 'add '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (mul-poly p1 p2))))
  (put 'sub '(polynomial 'polynomial)
       (lambda p1 p2) (tag (sub-poly p1 p2)))
  (put '=zero? 'polynomial (lambda (p) (terms-list-zero? 
                                        (term-list p))))
  (put 'negate 'polynomial (lambda (p) (tag (negate-poly p))))
  (put 'make 'polynomial
       (lambda (var terms)
         (tag (make-poly var terms))))
  'done)

;; dense representation of polynomials
;; list of coefficeients i.e. x^2 + 2x + 1 = '(1 2 1)
(define (install-dense-polynomial-package)
  ;;internal procedures
  ;; representation of poly
  (define (tag p) (attach-tag 'polynomial p))
  (define (make-poly variable term-list)
    (cons variable term-list))
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  
  ;;representation of terms and term lists
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (let* ((current-order (order (first-term term-list)))
               (new-order (order term))
               (order-diff (- (current-order new-order))))
          (cond ((= order-diff 1)
                 (cons (coeff term) term-list))
                ((> order-diff 1)
                 (cons (coeff term)
                       (append (zeroes-list (- order-diff 1))
                               (term-list))))
                (else
                 (error "TERM order greater must be greater than
                        highest order term in TERM-LIST:
                        ADJOIN-TERM"
                        (list term term-list)))))))
  (define (zeroes-list n)
    (if (= n 0)
        '()
        (cons 0 (zeroes-list (- n 1)))))
  
  (define (the-empty-termlist) '())
  (define (first-term term-list)
    (make-term 
     ;; first term is highest order
     (- (length term-list) 1)
     (car term-list)))
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list)
    (null? term-list))
  (define (make-term order coeff)
    (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (add-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               ADD-POLY"
               (list p1 p2))))
  
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1
                     (add-terms (rest-terms L1)
                                L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2
                     (add-terms
                      L1
                      (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term
                      (order t1)
                      (add (coeff t1)
                           (coeff t2)))
                     (add-terms
                      (rest-terms L1)
                      (rest-terms L2)))))))))
  
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (mul-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               MUL-POLY"
               (list p1 p2))))
  
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms
         (mult-term-by-all-terms
          (first-term L1) L2)
         (mul-terms (rest-terms L1) L2))))
  
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term
            (+ (order t1) (order t2))
            (mul (coeff t1) (ceoff t2)))
           (mul-term-by-all-terms
            t1
            (rest-terms L))))))
  
  (define (terms-list-zero? terms-list)
    (if (empty-termlist? terms-list)
        #t
        (and (=zero? (coeff (first-term terms-list)))
             (terms-list-zero (rest-terms terms-list)))))
  (define (negate-terms-list L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t (first-term L)))
          (adjoin-term
           (make-term (order term)
                      (negate (coeff term)))
           (negate-terms-list (rest-terms L))))))
  
  (define (negate-poly p)
    (make-poly (variable p)
               (negate-terms-list (term-list p))))
  (define (sub-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (add-terms (term-list p1)
                              (negate-terms-list p2)))
        (error "Polys not in same var:
               SUB-POLY"
               (list p1 p2))))

  ;;interface to rest of the system
  (define (tag p) (attach-tag 'polynomial p))
  (put 'add '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (mul-poly p1 p2))))
  (put 'sub '(polynomial 'polynomial)
       (lambda p1 p2) (tag (sub-poly p1 p2)))
  (put '=zero? 'polynomial (lambda (p) (terms-list-zero? 
                                        (term-list p))))
  (put 'negate 'polynomial (lambda (p) (tag (negate-poly p))))
  (put 'make 'polynomial
       (lambda (var terms)
         (tag (make-poly var terms))))
  'done)
;; ordered type hierarchy low->high 

(define (install-number-system)
  (install-type-tower '(integer rational real complex))
  (install-integer-package)
  (install-polar-package)
  (install-rectangular-package)
  (install-rational-package)
  (install-real-package)
  (install-complex-package)
  (install-sparse-polynomial-package)
  ;;or (install-dense-polynomial-package)
  )

;; (install-number-system)

### 2.90 Approach:

- User-facing procedures to create either representation of polynomial
- Make term-lists have a type
    - `('sparse ((3, 2) (1, 0)))`
    - `('dense (1, 0))`
    - Implement selectors for the term-list type and term list itself
    - Implement constructors for term-lists (attach type-tag to the list of terms)
- Internal generic `first-term` procedure to dispatch to the correct procedure for getting the first term based on the type of the term list
- Internal generic `adjoin-term` procedure to dispatch to the correct procedure for based on the type of term list
    - Dense term lists require padding with zeroes etc -> use procedure from previous exercise
- Ensure other procedures (`add`, `mul` etc) use the generic procedures for manipulating term lists

In [ ]:
;; procedures for users to make polynomials using either
;; sparse or dense representation

;; N.B No validation performed on the terms list, generic procedures
;; assume correct format for each type is passed in

;; terms = list of (order, coefficient) pairs
(define (make-sparse-polynomial var terms)
  ((get 'make '('polynomial 'sparse)) var terms))

;; terms = list of coefficients from high to low
(define (make-sparse-polynomial var terms)
  ((get 'make '('polynomial 'dense)) var terms))

(define (install-polynomial-package)
  ;; internal procedures
  ;; representation of poly
  (define (tag p) (attach-tag 'polynomial p))
    
  ;; poly constructors
  (define (make-sparse-poly variable terms)
    (attach-tag 'polynomial
                (make-poly variable
                           (make-term-list terms 'sparse))))
  (define (make-dense-poly variable terms)
    (attach-tag 'polynomial
                (make-poly variable
                           (make-term-list terms 'dense))))  

  (define (make-poly variable term-list)
    (cons variable term-list))
  
  (put 'make '('polynomial 'sparse) make-sparse-poly)
  (put 'make '('polynomial 'dense) make-dense-poly)
  
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  
  ;; representation of terms and term lists

  ;; internal generic term-list constructors
  (define (make-term-list terms type)
    ((get 'make-term-list type) terms))
  
  (put 'make-term-list 'sparse (lambda (terms) 
                                 (attach-tag 'sparse terms)))
  (put 'make-term-list 'dense (lambda (terms) 
                                 (attach-tag 'dense terms)))
  
  ;; internal term-list selectors
  (define (term-list-type term-list)
    (car term-list))
  (define (term-list-terms term-list)
    (cdr term-list))
  
  ;; internal generic first-term selectors
  (define (first-term term-list)
    ((get 'first-term (term-list-type term-list) term-list)))
  
  (define (first-term-sparse term-list)
    (car term-list))
  (define (first-term-dense term-list)
    (make-term 
     ;; first term is highest order
     (- (length term-list) 1)
     (car term-list)))
  
  (put 'first-term 'sparse first-term-sparse)
  (put 'first-term 'dense first-term-dense)
  
  ;; internal generic adjoin-term
  (define (adjoin-term term term-list)
    ((get 'adjoin-term (term-list-type term-list)) term term-list))
  
  (define (adjoin-term-sparse term term-list)
    (if (=zero? (coeff term))
        term-list
        (cons term term-list)))
  
  (define (adjoin-term-dense term term-list)
    (define (zeroes-list n)
      (if (= n 0)
          '()
          (cons 0 (zeroes-list (- n 1)))))
    (if (=zero? (coeff term))
        term-list
        (let* ((current-order (order (first-term term-list)))
               (new-order (order term))
               (order-diff (- (current-order new-order))))
          (cond ((= order-diff 1)
                 (cons (coeff term) term-list))
                ((> order-diff 1)
                 (cons (coeff term)
                       (append (zeroes-list (- order-diff 1))
                               (term-list))))
                (else
                 (error "TERM order greater must be greater than
                        highest order term in TERM-LIST:
                        ADJOIN-TERM"
                        (list term term-list)))))))
  
  (put 'adjoin-term 'sparse adjoin-term-sparse)
  (put 'adjoin-term 'dense adjoin-term-dense)
  
  (define (the-empty-termlist) '())
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list)
    (null? term-list))
  (define (make-term order coeff)
    (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (add-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               ADD-POLY"
               (list p1 p2))))
  
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1
                     (add-terms (rest-terms L1)
                                L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2
                     (add-terms
                      L1
                      (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term
                      (order t1)
                      (add (coeff t1)
                           (coeff t2)))
                     (add-terms
                      (rest-terms L1)
                      (rest-terms L2)))))))))
  
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1)
                        (variable p2))
        (make-poly
         (variable p1)
         (mul-terms (term-list p1)
                    (term-list p2)))
        (error "Polys not in same var:
               MUL-POLY"
               (list p1 p2))))
  
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms
         (mul-term-by-all-terms
          (first-term L1) L2)
         (mul-terms (rest-terms L1) L2))))
  
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term
            (+ (order t1) (order t2))
            (mul (coeff t1) (ceoff t2)))
           (mul-term-by-all-terms
            t1
            (rest-terms L))))))
  
  (define (terms-list-zero? terms-list)
    (if (empty-termlist? terms-list)
        #t
        (and (=zero? (coeff (first-term terms-list)))
             (terms-list-zero (rest-terms terms-list)))))
  
  (define (negate-terms-list L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t (first-term L)))
          (adjoin-term
           (make-term (order term)
                      (negate (coeff term)))
           (negate-terms-list (rest-terms L))))))
  
  (define (negate-poly p)
    (make-poly (variable p)
               (negate-terms-list (term-list p))))
  
  (define (sub-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (add-terms (term-list p1)
                              (negate-terms-list p2)))
        (error "Polys not in same var:
               SUB-POLY"
               (list p1 p2))))

  ;;interface to rest of the system
  (put 'add '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial)
       (lambda (p1 p2)
         (tag (mul-poly p1 p2))))
  (put 'sub '(polynomial 'polynomial)
       (lambda p1 p2) (tag (sub-poly p1 p2)))
  (put '=zero? 'polynomial (lambda (p) (terms-list-zero? 
                                        (term-list p))))
  (put 'negate 'polynomial (lambda (p) (tag (negate-poly p))))
  'done)

## 2.91

Univariate polynomial long division:
1. Divide highest order term of dividend by highest order term of divisor to get the first term of the quotient
2. Multiply result by the divisor
3. Subtract from dividend
4. Produce rest of answer recursively by dividing the difference by the divisor:
    - Stop when order of divisor exceeds order of dividend
        - Dividend = remainder
    - If dividend becomes zero -> quotient and remainder = 0

In [ ]:
(define (div-terms L1 L2)
  (if (empty-termlist? L1)
      (list (the-empty-termlist?)
            (the-empty-termlist))
      (let ((t1 (first-term L1))
            (t2 (first-term L2)))
        (if (> (order t2) (order t1))
            (list (the-empty-termlist) L1)
            (let* ((new-c (div (coeff t1)
                               (coeff t2)))
                   (new-o (- (order t1)
                             (order t2)))
                   (next-first-term (make-term new-o new-c)))
              (let ((rest-of-result
                     (div-terms
                      (sub-terms
                       L1
                       (mul-term-by-all-terms next-first-term L2))
                      L2)))
                (list
                 (adjoin-term next-first-term (car rest-of-result))
                 (cadr rest-of-result))))))))

(define (div-poly p1 p2)
  (if (same-variable? (variable p1)
                      (variable p2))
      (make-poly
       (variable p1)
       (div-terms (term-list p1)
                  (term-list p2)))
      (error "Polys not in same var:
             DIV-POLY"
             (list p1 p2))))